In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

In [18]:
def preprocess(train_path, test_path):
    input_texts = []
    target_texts = []
    
    test_input_texts = []
    test_target_texts = []
    
    num_lines_train_file = sum(1 for line in open(train_path, encoding='utf-8'))
    num_lines_test_file = sum(1 for line in open(test_path, encoding='utf-8'))
                    
    print("Read",train_path,"...")
    counter = 0
    with open(train_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            input_texts.append(data["english"])
            target_texts.append(data["chinese"])
            
            if(counter%1000000==0):
                print("Now processing {}/{} rows...".format(counter, num_lines_train_file))   
            
    print("Read",train_path,"finished!")
    
    print("\nRead",test_path,"...")
    counter = 0
    with open(test_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            
            if counter <= num_lines_test_file-100:
                input_texts.append(data["english"])
                target_texts.append(data["chinese"])
            else:
                test_input_texts.append(data["english"])
                test_target_texts.append(data["chinese"])
    print("Read",test_path,"finished!")      
          
    return input_texts, target_texts, test_input_texts, test_target_texts

In [ ]:
def getEncoderDecoderArray():
    

In [19]:
train_path = 'translation2019zh/translation2019zh_train.json'
test_path = 'translation2019zh/translation2019zh_valid.json'

input_texts, target_texts, test_input_texts, test_target_texts = preprocess(train_path,test_path)

Read translation2019zh/translation2019zh_train.json ...
Now processing 1000000/5161434 rows...
Now processing 2000000/5161434 rows...
Now processing 3000000/5161434 rows...
Now processing 4000000/5161434 rows...
Now processing 5000000/5161434 rows...
Read translation2019zh/translation2019zh_train.json finished!

Read translation2019zh/translation2019zh_valid.json ...
Read translation2019zh/translation2019zh_valid.json finished!


In [22]:
input_characters = sorted(list(set(input_texts)))
target_characters = sorted(list(set(target_texts)))

In [29]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 5200657
Number of unique input tokens: 5186506
Number of unique output tokens: 5114658
Max sequence length for inputs: 422
Max sequence length for outputs: 200


In [30]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')  

MemoryError: Unable to allocate 40.4 PiB for an array with shape (5200657, 422, 5186506) and data type float32